In [1]:
from ipywidgets import interact
from matplotlib import pyplot as plt
%matplotlib inline
import time

In [2]:
from sequence.kernel.timeline import Timeline
from sequence.topology.node import QuantumRouter, BSMNode
from sequence.components.optical_channel import QuantumChannel, ClassicalChannel

In [3]:
def test(sim_time, cc_delay, qc_atten, qc_dist):
    """
    sim_time: duration of simulation time (ms)
    cc_delay: delay on classical channels (ms)
    qc_atten: attenuation on quantum channels (dB/m)
    qc_dist: distance of quantum channels (km)
    """
    
    PS_PER_MS = 1e9  # Conversion factor for ms to ps
    M_PER_KM = 1e3   # Conversion factor for km to m
    
    # Convert units for cc_delay (to ps) and qc_dist (to m)
    cc_delay *= PS_PER_MS
    qc_dist *= M_PER_KM
    
    raw_fidelity = 0.85
    
    # Initialize the simulation timeline (argument in ps)
    tl = Timeline(sim_time * PS_PER_MS)
    #print(f"Timeline initialized for {sim_time} ms.")

    ## create our quantum network and update parameters as needed
    
    # first, construct the quantum routers
    # (with arguments for the node name, timeline, and number of quantum memories)
    
    
#EDITED QuantumRouter MEMORY VALUES - CONNOR
    NU = QuantumRouter("NU", tl, 100)
    StarLight = QuantumRouter("StarLight", tl, 100)
    UChicago_PME = QuantumRouter("UChicago_PME", tl, 100)
    UChicago_HC = QuantumRouter("UChicago_HC", tl, 75)#issues with changing to 100
    Fermilab_1 = QuantumRouter("Fermilab_1", tl, 100)
    Fermilab_2 = QuantumRouter("Fermilab_2", tl, 100)
    Argonne_1 = QuantumRouter("Argonne_1", tl, 100)
    Argonne_2 = QuantumRouter("Argonne_2", tl, 100)
    Argonne_3 = QuantumRouter("Argonne_3", tl, 150)#issues with changing to 100

    try:
        memory_array = UChicago_HC.get_components_by_type("MemoryArray")[0]
        print(f"UChicago_HC memory array initialized with {len(memory_array.memories)} memories.")
    except Exception as e:
        print(f"Error initializing UChicago_HC memories: {e}")

    try:
        memory_array = Argonne_3.get_components_by_type("MemoryArray")[0]
        print(f"Argonne_3 memory array initialized with {len(memory_array.memories)} memories.")
    except Exception as e:
        print(f"Error initializing Argonne_3 memories: {e}")

    print("Simulation setup complete.")
    
    # next, construct the BSM nodes
    # (with arguments for the node name, timeline, and the names of connected routers)
    
    m1 = BSMNode("m1", tl, ["NU", "StarLight"])
    m2 = BSMNode("m2", tl, ["StarLight", "UChicago_PME"])
    m3 = BSMNode("m3", tl, ["UChicago_PME", "UChicago_HC"])
    m4 = BSMNode("m4", tl, ["StarLight", "Fermilab_1"])
    m5 = BSMNode("m5", tl, ["StarLight", "Argonne_1"])
    m6 = BSMNode("m6", tl, ["UChicago_PME", "Argonne_1"])
    m7 = BSMNode("m7", tl, ["Fermilab_1", "Fermilab_2"])
    m8 = BSMNode("m8", tl, ["Argonne_1", "Argonne_2"])
    m9 = BSMNode("m9", tl, ["Argonne_1", "Argonne_3"])
    m10 = BSMNode("m10", tl, ["Fermilab_1", "Argonne_1"]) 

    NU.add_bsm_node(m1.name, StarLight.name)
    StarLight.add_bsm_node(m1.name, NU.name)

    StarLight.add_bsm_node(m2.name, UChicago_PME.name)
    UChicago_PME.add_bsm_node(m2.name, StarLight.name)

    UChicago_PME.add_bsm_node(m3.name, UChicago_HC.name)
    UChicago_HC.add_bsm_node(m3.name, UChicago_PME.name)

    StarLight.add_bsm_node(m4.name, Fermilab_1.name)
    Fermilab_1.add_bsm_node(m4.name, StarLight.name)

    StarLight.add_bsm_node(m5.name, Argonne_1.name)
    Argonne_1.add_bsm_node(m5.name, StarLight.name)

    UChicago_PME.add_bsm_node(m6.name, Argonne_1.name)
    Argonne_1.add_bsm_node(m6.name, UChicago_PME.name)

    Fermilab_1.add_bsm_node(m7.name, Fermilab_2.name)
    Fermilab_2.add_bsm_node(m7.name, Fermilab_1.name)

    Argonne_1.add_bsm_node(m8.name, Argonne_2.name)
    Argonne_2.add_bsm_node(m8.name, Argonne_1.name)

    Argonne_1.add_bsm_node(m9.name, Argonne_3.name)
    Argonne_3.add_bsm_node(m9.name, Argonne_1.name)

    Fermilab_1.add_bsm_node(m10.name, Argonne_1.name)  
    Argonne_1.add_bsm_node(m10.name, Fermilab_1.name)

    # set seeds for random generators
    nodes = [NU, StarLight, UChicago_PME, UChicago_HC, Fermilab_1, Fermilab_2, Argonne_1, Argonne_2, Argonne_3, m1, m2, m2, m4, m5, m6, m7, m8, m9, m10]
    for i, node in enumerate(nodes):
        node.set_seed(i)
        #print(f"Node {node.name} seed set to {i}.")

    for node in [NU, StarLight, UChicago_PME, UChicago_HC, Fermilab_1, Fermilab_2, Argonne_1, Argonne_2, Argonne_3]:
        memory_array = node.get_components_by_type("MemoryArray")[0]
        # update the coherence time (measured in seconds) here
        memory_array.update_memory_params("coherence_time", 10)
        # update the fidelity of entanglement for the memories
        memory_array.update_memory_params("raw_fidelity", raw_fidelity)

    # create all-to-all classical connections
    for node1 in nodes:
        for node2 in nodes:
            if node1 == node2:
                continue
            # construct a classical communication channel
            channel_name = f"cc_{node1.name}_{node2.name}"

            if channel_name in tl.entities:
                #print(f"Channel {channel_name} already exists. Skipping creation.")
                continue  # Skip creating this channel

            cc = ClassicalChannel(channel_name, tl, 1e3, delay=cc_delay)
            #print(f"Created Classical Channel: {channel_name}")
            cc.set_ends(node1, node2.name)


    # create quantum channels linking NU and StarLight to m1
    # (with arguments for the channel name, timeline, attenuation (in dB/m), and distance (in m))
    qc0 = QuantumChannel("qc_NU_m1", tl, qc_atten, qc_dist)
    qc1 = QuantumChannel("qc_StarLight_m1", tl, qc_atten, qc_dist)
    qc0.set_ends(NU, m1.name)
    qc1.set_ends(StarLight, m1.name)
    
    # create quantum channels linking StarLight and UChicago_PME to m2
    qc2 = QuantumChannel("qc_StarLight_m2", tl, qc_atten, qc_dist)
    qc3 = QuantumChannel("qc_UChicago_PME_m2", tl, qc_atten, qc_dist)
    qc2.set_ends(StarLight, m2.name)
    qc3.set_ends(UChicago_PME, m2.name)
    
    # create quantum channels linking UChicago_PME and UChicago_HC to m3
    qc4 = QuantumChannel("qc_UChicago_PME_m3", tl, qc_atten, qc_dist)
    qc5 = QuantumChannel("qc_UChicago_HC_m3", tl, qc_atten, qc_dist)
    qc4.set_ends(UChicago_PME, m3.name)
    qc5.set_ends(UChicago_HC, m3.name)
    
    # create quantum channels linking StarLight and Fermilab_1 to m4
    qc6 = QuantumChannel("qc_StarLight_m4", tl, qc_atten, qc_dist)
    qc7 = QuantumChannel("qc_Fermilab_1_m4", tl, qc_atten, qc_dist)
    qc6.set_ends(StarLight, m4.name)
    qc7.set_ends(Fermilab_1, m4.name)
    
    # create quantum channels linking StarLight and Argonne_1 to m5
    qc8 = QuantumChannel("qc_StarLight_m5", tl, qc_atten, qc_dist)
    qc9 = QuantumChannel("qc_Argonne_1_m5", tl, qc_atten, qc_dist)
    qc8.set_ends(StarLight, m5.name)
    qc9.set_ends(Argonne_1, m5.name)
    
    # create quantum channels linking UChicago_PME and Argonne_1 to m6
    qc10 = QuantumChannel("qc_UChicago_PME_m6", tl, qc_atten, qc_dist)
    qc11 = QuantumChannel("qc_Argonne_1_m6", tl, qc_atten, qc_dist)
    qc10.set_ends(UChicago_PME, m6.name)
    qc11.set_ends(Argonne_1, m6.name)
    
    # create quantum channels linking Fermilab_1 and Fermilab_2 to m7
    qc12 = QuantumChannel("qc_Fermilab_1_m7", tl, qc_atten, qc_dist)
    qc13 = QuantumChannel("qc_Fermilab_2_m7", tl, qc_atten, qc_dist)
    qc12.set_ends(Fermilab_1, m7.name)
    qc13.set_ends(Fermilab_2, m7.name)
    
    # create quantum channels linking Argonne_1 and Argonne_2 to m8
    qc14 = QuantumChannel("qc_Argonne_1_m8", tl, qc_atten, qc_dist)
    qc15 = QuantumChannel("qc_Argonne_2_m8", tl, qc_atten, qc_dist)
    qc14.set_ends(Argonne_1, m8.name)
    qc15.set_ends(Argonne_2, m8.name)
    
    # create quantum channels linking Argonne_1 and Argonne_3 to m9
    qc16 = QuantumChannel("qc_Argonne_1_m9", tl, qc_atten, qc_dist)
    qc17 = QuantumChannel("qc_Argonne_3_m9", tl, qc_atten, qc_dist)
    qc16.set_ends(Argonne_1, m9.name)
    qc17.set_ends(Argonne_3, m9.name)
    
    # create quantum channels linking Fermilab_1 and Argonne_1 to m10
    qc18 = QuantumChannel("qc_Fermilab_1_m10", tl, qc_atten, qc_dist)
    qc19 = QuantumChannel("qc_Argonne_1_m10", tl, qc_atten, qc_dist)
    qc18.set_ends(Fermilab_1, m10.name)
    qc19.set_ends(Argonne_1, m10.name)

    # create routing table manually
    # note that the routing table is based on quantum links, not classical
    # the arguments are the names of the destination node and the next node in the best path towards the destination
    
    # Routing table for NU
    NU.network_manager.protocol_stack[0].add_forwarding_rule("StarLight", "StarLight")
    NU.network_manager.protocol_stack[0].add_forwarding_rule("UChicago_PME", "StarLight")
    NU.network_manager.protocol_stack[0].add_forwarding_rule("UChicago_HC", "StarLight")
    NU.network_manager.protocol_stack[0].add_forwarding_rule("Fermilab_1", "StarLight")
    NU.network_manager.protocol_stack[0].add_forwarding_rule("Fermilab_2", "StarLight")
    NU.network_manager.protocol_stack[0].add_forwarding_rule("Argonne_1", "StarLight")
    NU.network_manager.protocol_stack[0].add_forwarding_rule("Argonne_2", "StarLight")
    NU.network_manager.protocol_stack[0].add_forwarding_rule("Argonne_3", "StarLight")
    
    # Routing table for StarLight
    StarLight.network_manager.protocol_stack[0].add_forwarding_rule("NU", "NU")
    StarLight.network_manager.protocol_stack[0].add_forwarding_rule("UChicago_PME", "UChicago_PME")
    StarLight.network_manager.protocol_stack[0].add_forwarding_rule("UChicago_HC", "UChicago_PME")
    StarLight.network_manager.protocol_stack[0].add_forwarding_rule("Fermilab_1", "Fermilab_1")
    StarLight.network_manager.protocol_stack[0].add_forwarding_rule("Fermilab_2", "Fermilab_1")
    StarLight.network_manager.protocol_stack[0].add_forwarding_rule("Argonne_1", "Argonne_1")
    StarLight.network_manager.protocol_stack[0].add_forwarding_rule("Argonne_2", "Argonne_1")
    StarLight.network_manager.protocol_stack[0].add_forwarding_rule("Argonne_3", "Argonne_1")
    
    # Routing table for UChicago_PME
    UChicago_PME.network_manager.protocol_stack[0].add_forwarding_rule("NU", "StarLight")
    UChicago_PME.network_manager.protocol_stack[0].add_forwarding_rule("StarLight", "StarLight")
    UChicago_PME.network_manager.protocol_stack[0].add_forwarding_rule("UChicago_HC", "UChicago_HC")
    UChicago_PME.network_manager.protocol_stack[0].add_forwarding_rule("Fermilab_1", "StarLight")
    UChicago_PME.network_manager.protocol_stack[0].add_forwarding_rule("Fermilab_2", "StarLight")
    UChicago_PME.network_manager.protocol_stack[0].add_forwarding_rule("Argonne_1", "Argonne_1")
    UChicago_PME.network_manager.protocol_stack[0].add_forwarding_rule("Argonne_2", "Argonne_1")
    UChicago_PME.network_manager.protocol_stack[0].add_forwarding_rule("Argonne_3", "Argonne_1")
    
    # Routing table for UChicago_HC
    UChicago_HC.network_manager.protocol_stack[0].add_forwarding_rule("NU", "UChicago_PME")
    UChicago_HC.network_manager.protocol_stack[0].add_forwarding_rule("StarLight", "UChicago_PME")
    UChicago_HC.network_manager.protocol_stack[0].add_forwarding_rule("UChicago_PME", "UChicago_PME")
    UChicago_HC.network_manager.protocol_stack[0].add_forwarding_rule("Fermilab_1", "UChicago_PME")
    UChicago_HC.network_manager.protocol_stack[0].add_forwarding_rule("Fermilab_2", "UChicago_PME")
    UChicago_HC.network_manager.protocol_stack[0].add_forwarding_rule("Argonne_1", "UChicago_PME")
    UChicago_HC.network_manager.protocol_stack[0].add_forwarding_rule("Argonne_2", "UChicago_PME")
    UChicago_HC.network_manager.protocol_stack[0].add_forwarding_rule("Argonne_3", "UChicago_PME")
    
    # Routing table for Fermilab_1
    Fermilab_1.network_manager.protocol_stack[0].add_forwarding_rule("NU", "StarLight")
    Fermilab_1.network_manager.protocol_stack[0].add_forwarding_rule("StarLight", "StarLight")
    Fermilab_1.network_manager.protocol_stack[0].add_forwarding_rule("UChicago_PME", "StarLight")
    Fermilab_1.network_manager.protocol_stack[0].add_forwarding_rule("UChicago_HC", "StarLight")
    Fermilab_1.network_manager.protocol_stack[0].add_forwarding_rule("Fermilab_2", "Fermilab_2")
    Fermilab_1.network_manager.protocol_stack[0].add_forwarding_rule("Argonne_1", "Argonne_1")
    Fermilab_1.network_manager.protocol_stack[0].add_forwarding_rule("Argonne_2", "Argonne_1")
    Fermilab_1.network_manager.protocol_stack[0].add_forwarding_rule("Argonne_3", "Argonne_1")
    
    # Routing table for Fermilab_2
    Fermilab_2.network_manager.protocol_stack[0].add_forwarding_rule("NU", "Fermilab_1")
    Fermilab_2.network_manager.protocol_stack[0].add_forwarding_rule("StarLight", "Fermilab_1")
    Fermilab_2.network_manager.protocol_stack[0].add_forwarding_rule("UChicago_PME", "Fermilab_1")
    Fermilab_2.network_manager.protocol_stack[0].add_forwarding_rule("UChicago_HC", "Fermilab_1")
    Fermilab_2.network_manager.protocol_stack[0].add_forwarding_rule("Fermilab_1", "Fermilab_1")
    Fermilab_2.network_manager.protocol_stack[0].add_forwarding_rule("Argonne_1", "Fermilab_1")
    Fermilab_2.network_manager.protocol_stack[0].add_forwarding_rule("Argonne_2", "Fermilab_1")
    Fermilab_2.network_manager.protocol_stack[0].add_forwarding_rule("Argonne_3", "Fermilab_1")
    
    # Routing table for Argonne_1
    Argonne_1.network_manager.protocol_stack[0].add_forwarding_rule("NU", "StarLight")
    Argonne_1.network_manager.protocol_stack[0].add_forwarding_rule("StarLight", "StarLight")
    Argonne_1.network_manager.protocol_stack[0].add_forwarding_rule("UChicago_PME", "UChicago_PME")
    Argonne_1.network_manager.protocol_stack[0].add_forwarding_rule("UChicago_HC", "UChicago_PME")
    Argonne_1.network_manager.protocol_stack[0].add_forwarding_rule("Fermilab_1", "Fermilab_1")
    Argonne_1.network_manager.protocol_stack[0].add_forwarding_rule("Fermilab_2", "Fermilab_1")
    Argonne_1.network_manager.protocol_stack[0].add_forwarding_rule("Argonne_2", "Argonne_2")
    Argonne_1.network_manager.protocol_stack[0].add_forwarding_rule("Argonne_3", "Argonne_3")
    
    # Routing table for Argonne_2
    Argonne_2.network_manager.protocol_stack[0].add_forwarding_rule("NU", "Argonne_1")
    Argonne_2.network_manager.protocol_stack[0].add_forwarding_rule("StarLight", "Argonne_1")
    Argonne_2.network_manager.protocol_stack[0].add_forwarding_rule("UChicago_PME", "Argonne_1")
    Argonne_2.network_manager.protocol_stack[0].add_forwarding_rule("UChicago_HC", "Argonne_1")
    Argonne_2.network_manager.protocol_stack[0].add_forwarding_rule("Fermilab_1", "Argonne_1")
    Argonne_2.network_manager.protocol_stack[0].add_forwarding_rule("Fermilab_2", "Argonne_1")
    Argonne_2.network_manager.protocol_stack[0].add_forwarding_rule("Argonne_1", "Argonne_1")
    Argonne_2.network_manager.protocol_stack[0].add_forwarding_rule("Argonne_3", "Argonne_3")
    
    # Routing table for Argonne_3
    Argonne_3.network_manager.protocol_stack[0].add_forwarding_rule("NU", "Argonne_1")
    Argonne_3.network_manager.protocol_stack[0].add_forwarding_rule("StarLight", "Argonne_1")
    Argonne_3.network_manager.protocol_stack[0].add_forwarding_rule("UChicago_PME", "Argonne_1")
    Argonne_3.network_manager.protocol_stack[0].add_forwarding_rule("UChicago_HC", "Argonne_1")
    Argonne_3.network_manager.protocol_stack[0].add_forwarding_rule("Fermilab_1", "Argonne_1")
    Argonne_3.network_manager.protocol_stack[0].add_forwarding_rule("Fermilab_2", "Argonne_1")
    Argonne_3.network_manager.protocol_stack[0].add_forwarding_rule("Argonne_1", "Argonne_1")
    Argonne_3.network_manager.protocol_stack[0].add_forwarding_rule("Argonne_2", "Argonne_2")

    ## run our simulation
    
    tl.init()
    
    # Entanglement requests between all nodes in the network
    # we use the network manager of an end router to make our entanglement request
    # here, the arguments are:
    # (1) the destination node name,
    # (2) the start time (in ps) of entanglement,
    # (3) the end time (in ps) of entanglement,
    # (4) the number of memories to entangle, and
    # (5) the desired fidelity of entanglement.
    
    # NU requests entanglement with StarLight
    NU.network_manager.request("StarLight", 1e12, 1e14, 50, 0.9)
    
    # StarLight requests entanglement with UChicago_PME
    StarLight.network_manager.request("UChicago_PME", 1e12, 1e14, 50, 0.9)
    
    # UChicago_PME requests entanglement with UChicago_HC
    UChicago_PME.network_manager.request("UChicago_HC", 1e12, 1e14, 50, 0.9)
    
    # UChicago_HC requests entanglement with Fermilab_1
    UChicago_HC.network_manager.request("Fermilab_1", 1e12, 1e14, 50, 0.9)
    
    # Fermilab_1 requests entanglement with Fermilab_2
    Fermilab_1.network_manager.request("Fermilab_2", 1e12, 1e14, 50, 0.9)
    
    # Fermilab_2 requests entanglement with Argonne_1
    Fermilab_2.network_manager.request("Argonne_1", 1e12, 1e14, 50, 0.9)
    
    # Argonne_1 requests entanglement with Argonne_2
    Argonne_1.network_manager.request("Argonne_2", 1e12, 1e14, 50, 0.9)
    
    # Argonne_2 requests entanglement with Argonne_3
    Argonne_2.network_manager.request("Argonne_3", 1e12, 1e14, 50, 0.9)
    
    # Argonne_3 requests entanglement with NU 
    Argonne_3.network_manager.request("NU", 1e12, 1e14, 50, 0.9)
    
    tick = time.time()
    tl.run()
    #print("execution time %.2f sec" % (time.time() - tick))

    #### display metrics for entangled memories ####

    fig, axs = plt.subplots(3, 3)  
    fig.set_size_inches(15, 15)
    
    routers = [NU, StarLight, UChicago_PME, UChicago_HC, Fermilab_1, Fermilab_2, Argonne_1, Argonne_2, Argonne_3]
    titles = ["NU", "StarLight", "UChicago_PME", "UChicago_HC", "Fermilab_1", "Fermilab_2", "Argonne_1", "Argonne_2", "Argonne_3"]
    
    # Loop through each router and corresponding subplot to plot the number of entangled memories over time
    for i, (router, title) in enumerate(zip(routers, titles)):
        data = []
        for info in router.resource_manager.memory_manager:
            if info.entangle_time > 0:
                data.append(info.entangle_time / 1e12)  # Convert time to seconds
        data.sort()
        
        ax = axs[i // 3, i % 3]  # Position in the 3x3 grid
        ax.plot(data, range(1, len(data) + 1), marker="o")
        ax.set_title(title)
        ax.set_ylabel("Number of Entangled Memories")
        if i // 3 == 2:
            ax.set_xlabel("Simulation Time (s)")  # Add x-label only on the last row
        
    fig.tight_layout()
    plt.show()

    ## display metrics for memory fidelities
    
    fig, axs = plt.subplots(3, 3)  # Create a 3x3 grid for nine routers
    fig.set_size_inches(15, 15)
    
    routers = [NU, StarLight, UChicago_PME, UChicago_HC, Fermilab_1, Fermilab_2, Argonne_1, Argonne_2, Argonne_3]
    titles = ["NU", "StarLight", "UChicago_PME", "UChicago_HC", "Fermilab_1", "Fermilab_2", "Argonne_1", "Argonne_2", "Argonne_3"]
    
    # Loop through each router and corresponding subplot to plot memory fidelity
    for i, (router, title) in enumerate(zip(routers, titles)):
        data = []
        for info in router.resource_manager.memory_manager:
            data.append(info.fidelity)
        
        ax = axs[i // 3, i % 3]  
        ax.bar(range(len(data)), data)  
        ax.plot([0, len(data)], [raw_fidelity, raw_fidelity], "k--")  
        ax.plot([0, len(data)], [0.9, 0.9], "k--")  
        ax.set_ylim(0.7, 1)  
        ax.set_title(title)  
        
        if i % 3 == 0:  
            ax.set_ylabel("Fidelity")
        if i // 3 == 2:  
            ax.set_xlabel("Memory Number")
    
    fig.tight_layout()
    plt.show()





In [4]:
interactive_plot = interact(test, sim_time=(2000, 4000, 500), cc_delay=(0.1, 1, 0.1), qc_atten=[1e-5, 2e-5, 3e-5], qc_dist=(1, 10, 1))
interactive_plot

from ipywidgets import interact

interactive(children=(IntSlider(value=3000, description='sim_time', max=4000, min=2000, step=500), FloatSlider…